In [ ]:
# --- Continuação do script anterior ---
# Assumimos que o DataFrame 'df_agregado' da Fase 1 já foi criado.

import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import plotly.graph_objects as go

# --- FUNÇÕES PARA A FASE 2 ---

def carregar_dados_referencia(caminho_arquivo_csv, fuso_horario='America/Sao_Paulo'):
    """
    Carrega, limpa e prepara os dados de uma estação meteorológica oficial.

    Args:
        caminho_arquivo_csv (str): O caminho para o arquivo .csv baixado.
        fuso_horario (str): O fuso horário original dos dados da estação oficial.

    Returns:
        pd.DataFrame: DataFrame com os dados de referência, limpo e com fuso horário UTC.
    """
    print(f"\n--- Carregando dados de referência de: {caminho_arquivo_csv} ---")
    try:
        # Carrega o CSV. NOTA: Pode ser necessário ajustar separadores (sep=';') ou cabeçalhos (header=)
        # dependendo do formato do arquivo da Epagri/INMET.
        df_ref = pd.read_csv(caminho_arquivo_csv, sep=';', header=10, decimal=',')

        # --- Limpeza e Renomeação (AJUSTE CONFORME O ARQUIVO REAL) ---
        # Esta parte é um exemplo e precisará ser adaptada aos nomes reais das colunas do seu CSV.
        df_ref['timestamp'] = pd.to_datetime(df_ref['Data'] + ' ' + df_ref['Hora (UTC)'])
        df_ref = df_ref.rename(columns={'Temp. [°C]': 'temp_referencia'})
        df_ref = df_ref[['timestamp', 'temp_referencia']]

        # Converte a coluna de tempo para o fuso horário correto e depois para UTC para compatibilidade
        df_ref['timestamp'] = df_ref['timestamp'].dt.tz_localize(fuso_horario).dt.tz_convert('UTC')

        # Define o timestamp como índice
        df_ref = df_ref.set_index('timestamp')
        df_ref['temp_referencia'] = pd.to_numeric(df_ref['temp_referencia'], errors='coerce')

        print("Dados de referência carregados e preparados com sucesso.")
        return df_ref

    except FileNotFoundError:
        print(f"ERRO: Arquivo não encontrado em '{caminho_arquivo_csv}'. Verifique o caminho.")
        return None
    except Exception as e:
        print(f"Ocorreu um erro ao carregar os dados de referência: {e}")
        return None


def sincronizar_dados(df_sensores_agregado, df_referencia, intervalo='1H'):
    """
    Sincroniza os dados dos nossos sensores com os da estação de referência em um intervalo de tempo comum.
    """
    print(f"\n--- Sincronizando dados em janelas de {intervalo} ---")

    # Reamostra os dados dos nossos sensores para o intervalo desejado (ex: 1 hora)
    df_sensores_sync = df_sensores_agregado.xs('mean', level=1, axis=1).resample(intervalo).mean()

    # Reamostra os dados da estação de referência para o mesmo intervalo
    df_referencia_sync = df_referencia.resample(intervalo).mean()

    # Junta os dois DataFrames com base no índice de tempo
    df_final_comparativo = pd.merge(df_sensores_sync, df_referencia_sync, left_index=True, right_index=True, how='inner')

    # Remove linhas onde a referência ou todos os sensores são nulos
    df_final_comparativo.dropna(subset=['temp_referencia'], inplace=True)

    print("Dados sincronizados com sucesso. DataFrame comparativo final criado.")
    return df_final_comparativo


# --- BLOCO DE EXECUÇÃO DA FASE 2 ---

# Exemplo de como usar as funções:
# 1. Faça o download do arquivo CSV da Epagri ou INMET e coloque o nome do arquivo abaixo.
nome_do_arquivo_oficial = 'dados_estacao_sao_jose.csv'
df_referencia = carregar_dados_referencia(nome_do_arquivo_oficial)

if df_referencia is not None:
    # 2. Sincronize os dados (assumindo que 'df_agregado' existe da Fase 1)
    df_comparativo = sincronizar_dados(df_agregado, df_referencia, intervalo='1H')

    # 3. Exibe o resultado final, pronto para o cálculo de erros
    print("\n--- DataFrame Sincronizado para Análise Comparativa ---")
    display(df_comparativo.head())
